In [1]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 24.9 MB/s eta 0:00:00


# A TransShipment Problem

![slide1.png](https://i.ibb.co/XLWcpmr/download.png)


In [2]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb

To define our node id's we use a python generator to generate a number for each node

In [3]:
N = [str(i+1) for i in range(6)]
N

['1', '2', '3', '4', '5', '6']

Now we provide the supply/demand in the sequence of our node ids.

In [4]:
supply = [5, 2, 0, -2, -4, -1]

Just to make sure we generated a label for each node and put in a supply for each node we include an assert.

In [5]:
assert len(supply) == len(N)

We can now convert this into a dictionary of values, with node IDs used as keys.

In [6]:
D = dict(zip(N, supply))
D

{'1': 5, '2': 2, '3': 0, '4': -2, '5': -4, '6': -1}

We give the cost of each link as a dictionary with a tuple (origin, destination). The cost from Node o to Node d can be retrieved with C\[(o,d)]

In [7]:
C = {('1', '2'): 1,
     ('1', '3'): 4,
     ('2', '3'): 2,
     ('3', '4'): 2,
     ('3', '5'): 5,
     ('3', '6'): 8,
     ('4', '5'): 1}

Similarily we list the capacity of each link.

In [8]:
CAPACITY = { ('1', '2'): 3,
             ('1', '3'): 3,
             ('2', '3'): 7,
             ('3', '4'): 5,
             ('3', '5'): 7,
             ('3', '6'): 1,
             ('4', '5'): 3}

In [9]:
#Just to check if we did not make an input error we do a quick assert on the length of both input dicts
assert len(CAPACITY) == len(C)

First we create our gurobi model:

In [10]:
# Define the model
m = gp.Model('TransShipmentProblem')

Restricted license - for non-production use only - expires 2024-10-28


And finally, we can declare our decision variables: for each arc we define the flow. By setting the data-type with vtype as an Integer we restrict the decision variables to be 0 or higher.

In [11]:
x = m.addVars(C, vtype=GRB.INTEGER, name='x')
x

{('1', '2'): <gurobi.Var *Awaiting Model Update*>,
 ('1', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('2', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '4'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('4', '5'): <gurobi.Var *Awaiting Model Update*>}

## Objective and constraints
The objective function which we want to minimize is the cost of the flow in the network.
Thus, the sum over the cost of each arc multiplied by the flow variable for the arc.

In [12]:
#Set the objective function as the product of the flow number of each arch multipled by the cost of that arc.
m.setObjective(x.prod(C), GRB.MINIMIZE)

Next we specify our flow conservation constraint in the network. For each node i in the system, we constrain the difference between the incoming flow and outgoing flow to be equal to the specified supply for that node.

In [13]:
for i in N:
    m.addConstr(x.sum(i,'*') - x.sum('*',i) == D[i], name=f'FlowConstraints_{i}')

To constrain each arc at its capacity, we iterate through all the decision variables and add a constraint for each.

In [14]:
for (o, d), var in x.items():
    m.addConstr(var <= CAPACITY[(o,d)], name=f"CapacityConstraints_{o}_{d}")

Write Gurobi model to text file for inspection

In [16]:
# Write Gurobi model to file for inspection
m.write('TransShipment.lp')

In [31]:
# Print text file
with open('TransShipment.lp') as f:
  print (f.read())

\ Model TransShipmentProblem
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  x[1,2] + 4 x[1,3] + 2 x[2,3] + 2 x[3,4] + 5 x[3,5] + 8 x[3,6] + x[4,5]
Subject To
 FlowConstraints_1: x[1,2] + x[1,3] = 5
 FlowConstraints_2: - x[1,2] + x[2,3] = 2
 FlowConstraints_3: - x[1,3] - x[2,3] + x[3,4] + x[3,5] + x[3,6] = 0
 FlowConstraints_4: - x[3,4] + x[4,5] = -2
 FlowConstraints_5: - x[3,5] - x[4,5] = -4
 FlowConstraints_6: - x[3,6] = -1
 CapacityConstraints_1_2: x[1,2] <= 3
 CapacityConstraints_1_3: x[1,3] <= 3
 CapacityConstraints_2_3: x[2,3] <= 7
 CapacityConstraints_3_4: x[3,4] <= 5
 CapacityConstraints_3_5: x[3,5] <= 7
 CapacityConstraints_3_6: x[3,6] <= 1
 CapacityConstraints_4_5: x[4,5] <= 3
Bounds
Generals
 x[1,2] x[1,3] x[2,3] x[3,4] x[3,5] x[3,6] x[4,5]
End



In [18]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 13 rows, 7 columns and 21 nonzeros
Model fingerprint: 0xa0573156
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 13 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 47 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.700000000000e+01, best bound 4.700000000000e+01, gap 0.0000%


In [19]:
print(f"Optimal objective value: {m.objVal}")

print("\nShipment plan:")
for o, d in x.keys():
    if (abs(x[o, d].x) > 0): #Only print if not 0
        print (f"Ship {x[o,d].x} units from node {o} to node {d}")

Optimal objective value: 47.0

Shipment plan:
Ship 3.0 units from node 1 to node 2
Ship 2.0 units from node 1 to node 3
Ship 5.0 units from node 2 to node 3
Ship 5.0 units from node 3 to node 4
Ship 1.0 units from node 3 to node 5
Ship 1.0 units from node 3 to node 6
Ship 3.0 units from node 4 to node 5


Our mimimal transport cost is 47 and we have the flow for each arc given by the decision variables.

![slide2.png](https://i.ibb.co/wWYbzJw/download-1.png)

## Second Transhipment point

We have the opportunity to place a second transshipment point that is more centrally located. However there is a much higher fixed cost attached for using this point. Are the lower transport costs worth this?

In [20]:
#New transport cost OD matrix, with added transshipment point 7.
#Lower transport costs to stores, higher from factories.
C = {('1', '2'): 1,
     ('1', '3'): 4,
     ('1', '7'): 5,
     ('2', '3'): 2,
     ('2', '7'): 3,
     ('3', '4'): 2,
     ('3', '5'): 5,
     ('3', '6'): 8,
     ('4', '5'): 1,
     ('7', '4'): 1,
     ('7', '5'): 1,
     ('7', '6'): 7}


N = [str(i+1) for i in range(7)]
N

supply = [5, 2, 0, -2, -4, -1, 0]

D = dict(zip(N, supply))
D

#New capacity OD matrix, with added transshipment point 7
CAPACITY = { ('1', '2'): 3,
             ('1', '3'): 3,
             ('1', '7'): 3,
             ('2', '3'): 7,
             ('2', '7'): 7,
             ('3', '4'): 5,
             ('3', '5'): 7,
             ('3', '6'): 1,
             ('4', '5'): 3,
             ('7', '4'): 5,
             ('7', '5'): 7,
             ('7', '6'): 1}

#Cost for using a node
node_costs = {'7' : 4, '3' : 1}

E = [(i,j) for i in N for j in N if i in C.keys() if j in C[i].keys()]

In [21]:
# Define our new model
m = gp.Model('TransShipmentProblemPlus')

In [22]:
x = m.addVars(C, vtype=GRB.INTEGER, name='x')
x

{('1', '2'): <gurobi.Var *Awaiting Model Update*>,
 ('1', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('1', '7'): <gurobi.Var *Awaiting Model Update*>,
 ('2', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('2', '7'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '4'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('3', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('4', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('7', '4'): <gurobi.Var *Awaiting Model Update*>,
 ('7', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('7', '6'): <gurobi.Var *Awaiting Model Update*>}

Add a new binary variable that indices whether we use transhippment place 3 or 7

In [23]:
y = m.addVars(node_costs, vtype=GRB.BINARY, name='USE_TRANSSHIPMENT')
y

{'7': <gurobi.Var *Awaiting Model Update*>,
 '3': <gurobi.Var *Awaiting Model Update*>}

In [24]:
#Set the objective function as the product of the flow number of each arch multipled by the cost of that arc.
# Note that we now also added the unit costs of running either transhipment point 3 or 7
m.setObjective(y.prod(node_costs) + x.prod(C), GRB.MINIMIZE)

Next we will again specify our flow conservation constraint in the network. For each node i in the system, we constrain the difference between the incoming flow and outgoing flow to be equal to the specified supply for that node.

In [25]:
for i in N:
    m.addConstr(x.sum(i,'*') - x.sum('*',i) == D[i], name=f'FlowConstraints_{i}')

To again constrain each arc at its capacity, we iterate through all the decision variables and add a constraint for each.

In [26]:
for (o, d), var in x.items():
    m.addConstr(var <= CAPACITY[(o,d)], name=f"CapacityConstraints_{o}_{d}")

In [27]:
network_capacity = sum(CAPACITY.values())
#Adding linking constraints to limit the in and outgoing flow to 0, if that node is not being used.

m.addConstrs((x.sum(node) - y[node]*network_capacity <= 0 for node in y.keys()), name='LinkingIn')
m.addConstrs((x.sum('*',node) - y[node]*network_capacity <= 0 for node in y.keys()), name='LinkingOut')

{'7': <gurobi.Constr *Awaiting Model Update*>,
 '3': <gurobi.Constr *Awaiting Model Update*>}

In [28]:
y

{'7': <gurobi.Var *Awaiting Model Update*>,
 '3': <gurobi.Var *Awaiting Model Update*>}

In [29]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 23 rows, 14 columns and 50 nonzeros
Model fingerprint: 0xa87ebe46
Variable types: 0 continuous, 14 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 49.0000000
Presolve removed 13 rows and 1 columns
Presolve time: 0.00s
Presolved: 10 rows, 13 columns, 36 nonzeros
Variable types: 0 continuous, 13 integer (3 binary)

Root relaxation: objective 4.337500e+01, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   43.37500    0    2   49.00000  

In [30]:
print(f"Optimal objective value: {m.objVal}")

for n, var in y.items():
    if (abs(var.x) > 0): #Only print if not 0
        print (f"Use transhipment point {n}")
print ("Shipment plan")
for (o,d), var in x.items():
    if (abs(var.x) > 0): #Only print if not 0
        print (f"Ship {x[o,d].x} units from node {o} to node {d}")

Optimal objective value: 45.0
Use transhipment point 7
Shipment plan
Ship 3.0 units from node 1 to node 2
Ship 2.0 units from node 1 to node 7
Ship 5.0 units from node 2 to node 7
Ship 2.0 units from node 7 to node 4
Ship 4.0 units from node 7 to node 5
Ship 1.0 units from node 7 to node 6


Running this model indicates that a solution with only node 7 has the lowest transport costs.
This indicates that renting the space for more centrally located transhipment will pay itself back in savings on transport costs.



---

**Acknowledgements**

The first Transshipment Problem was originally authored for SCM.250 by Dr. Josue Velasquez
All problems modified to Gurobi in Python by Dr. Elenna Dugundji & Dr. Thomas Koch, CTL, 2023, CC-BY-SA
https://creativecommons.org/licenses/by-sa/3.0/nz/